#### Load Resolved Aliases into Database

In [1]:
import sys
import mariadb
import time
import hashlib
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from kitchen_sink_class import Query
from kitchen_sink_class import load_single_line_from_file as load_it
if 1 == 0:
    q = Query()
    q.preload_alias_maps();

    pwd = load_it('./db.pwd')
    mydb = mariadb.connect(
        port=13306,
        host='127.0.0.1',
        user='w3hacknet',
        password=pwd,
        database='w3hacknet',
        autocommit=True
    )

    cursor = mydb.cursor()

    for key in q.hackers.keys():
        val = q.hackers[key]
        for v in val:
            cursor.callproc('w3hacknet.resolveAlias', (hashlib.md5(v.encode('utf-8')).hexdigest(), key))

    print('done!')
    #raise Exception('Do not continue!')

#### Spider Mechanics by resolved Alias

In [ ]:
import sys
import mariadb
import time
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from kitchen_sink_class import Query
from kitchen_sink_class import load_single_line_from_file as load_it
class MyRepoCounter:
    def __init__(self, repo_dict):
        self.repo_name = repo_dict['name']
        self.repo_full_name = repo_dict['full_name']
        owner = repo_dict['owner']
        self.owner = owner['login']
        self.count = 0
    def add_one(self):
        self.count += 1
        
    def key(self):
        return self.repo_full_name


q = Query()

pwd = load_it('./db.pwd')
mydb = mariadb.connect(
    port=3306,
    host='127.0.0.1',
    user='w3hacknet',
    password=pwd,
    database='w3hacknet',
    autocommit=True
)

cursor = mydb.cursor()
cursor.execute('select now();')
print(cursor.fetchone())

repo_counter = {}
call_count = 0
running = True
owner = {}
whoami = 'matt'
user_id = ''
alias_hash = ''
while running:
    
    cursor.callproc('w3hacknet.reserveNextUser', 
                    (whoami, user_id, alias_hash))
    if cursor.sp_outparams:
        tup = cursor.fetchone()
        user_id = tup[0]
        alias_hash = tup[1]
    else:
        raise Exception('Abort abort abort!')
    print('Working on', user_id, alias_hash)
    done = False
    q.reset_last_date()
    last_count = -1
    while not done:
        time.sleep(3) # Don't over stay our welcome - Can't exceed 3600/hr, let alone 5000
        body = q.load_hacker_url(user_id)
        call_count += 1
        if call_count % 25 == 0:
            print(call_count, 'rest API calls made')

        if (body == None):
            print('Unable to load JSON')
            done = True
        else:
            total_count = body['total_count']
            if (total_count == last_count):
                print('Identical result set found.  Moving on.', total_count, last_count)
                done = True
            else:
                print(total_count, 'remaining commits for user', user_id, 'Repo_count', len(repo_counter))
            last_count = total_count
            if total_count > 20000:
                print('Yikes!', total_count, ' seems like a few too many')
                done = True
            incomplete_results = body['incomplete_results']
#            print(total_count)
#            print(incomplete_results)
            array = body['items'];
            if (array == None or len(array) < 1):
                done = True
            else:
                for n in array:
                    commit = n['commit']
                    comAuth = commit['author']
                    commit_date = q.set_last_date(comAuth['date'])

                    repo = n['repository']
                    repo_full_name = repo['full_name']
                    counter = None
                    if repo_full_name not in repo_counter:
                        counter = MyRepoCounter(repo)
                        repo_counter[repo_full_name] = counter
                    else:
                        counter = repo_counter[repo_full_name]
                    counter.add_one()
                    cursor.callproc('w3hacknet.addUpdateRepo', 
                           (counter.owner, counter.repo_name, commit_date))

                if (total_count < 100 and incomplete_results == False):
                    done = True


In [ ]:
'''
last_user = 'nobody'
for k in repo_counter.keys():
    cursor.callproc('w3hacknet.setRepoCount', 
       (counter.owner, counter.repo_name, counter.count))
print('done')
'''

In [3]:
class Boob:
    def __init__(self):
        self.name = 'Ralph'
    def my_func(self):
        return self.name

class Buntu(Boob):
    def __init__(self):
        super().__init__()
    def print_it(self):
        print(self.my_func())

b = Buntu()
b.print_it()

Ralph


In [12]:
import json
import time
from os.path import exists
p = '../../../active/contf.json'
garbage = True
while garbage:
    if exists(p):
        with open(p, 'rb') as reader:
            thingy = json.load(reader)
        print('Loaded ', thingy[99]['total'])
        time.sleep(0)
        garbage = False
    else:
        print('try again')
        exit(0)

Loaded  13


In [ ]:
import hashlib
hashlib.md5('Ethan Buchman <ethan@coinculture.info>'.encode('utf-8')).hexdigest()